In [1]:
from bs4 import BeautifulSoup
import lxml
import requests
import os

from urllib import parse

## 파이썬으로 크롤링하기

pip > pypi > pipenv > library

### pyenv 가상환경 설정

pyenv virtualenv 3.6.5 <가상환경명>

pyenv local <가상환경명>

### requests 모듈

파이썬으로 HTTP get 요청을 하기 위한 모듈

In [2]:
url = 'https://comic.naver.com/webtoon/weekday.nhn'

headers = {
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X x.y; rv:10.0) Gecko/20100101 Firefox/10.0'
}
res = requests.get(url, headers=headers)


In [3]:
# f = open('weekday.html', 'wt')
# f.write(res.text)
# f.close()

# with open('weekday.html', 'wt') as f:
#     f.write(res.text)

In [4]:
# html_doc = """
# <html><head><title>The Dormouse's story</title></head>
# <body>
# <p class="title"><b>The Dormouse's story</b></p>

# <p class="story">Once upon a time there were three little sisters; and their names were
# <a href="http://example.com/elsie" class="sister" id="link1">Elsie</a>,
# <a href="http://example.com/lacie" class="sister" id="link2">Lacie</a> and
# <a href="http://example.com/tillie" class="sister" id="link3">Tillie</a>;
# and they lived at the bottom of a well.</p>

# <p class="story">...</p>
# """

In [5]:
# html = open('weekday.html', 'rt').read()
# soup = BeautifulSoup(html, 'lxml')

# div_content = soup.find('div', id='content')
# div_list_area = div_content.find('div', class_='list_area')
# div_col_list = div_list_area.find_all('div', class_='col')

# for col in div_col_list:
#     print(col.get_text())


In [6]:
# file_path = 'weekday.html'
# url_episode_list = 'https://comic.naver.com/webtoon/list.nhn'
# params = {
#     'titleId': 703845,
# }
# if os.path.exists(file_path):
#     html = open(file_path, 'rt').read()
# else:
#     response = requests.get(url_episode_list, params)
#     html = response.text
#     open(file_path, 'wt').write(html)

# soup = BeautifulSoup(html, 'lxml')

# h2_title = soup.select_one('div.detail > h2')
# title = h2_title.contents[0].strip()
# author = h2_title.contents[1].get_text(strip=True)
# # desc = soup.select_one('div.detail > h2').next_sibling.next_element
# desc = soup.select_one('div.detail > p').get_text()

# print(desc)
# print(title)
# print(author)

In [7]:
# table = soup.select_one('table.viewList')


# tr_list = table.select('tr')

# for index, tr in enumerate(tr_list[1:]):
#     if tr.get('class'):
#         continue
        
#     url_thumnail = tr.select_one('td:nth-of-type(1) img').get('src')
#     url_detail = tr.select_one('td > a').get('href')
    
#     query_string = parse.urlsplit(url_detail).query
#     query_dict = parse.parse_qs(query_string)
    
#     no = query_dict['no'][0]
    
#     title = tr.select_one('td:nth-of-type(2) > a').get_text(strip=True)
#     rating = tr.select_one('td:nth-of-type(3) strong').get_text(strip=True)
#     created_data = tr.select_one('td:nth-of-type(4)').get_text(strip=True)
    
# #     print(detail)
#     print(no)
# #     print(url_thumnail)
# #     print(url_detail)
# #     print(title)
# #     print(rating)
# #     print(created_data)


In [19]:
class Webtoon:
    def __init__(self, webtoon_id):
        self.webtoon_id = webtoon_id
        info = self.get_info()
        self.title = info['title']
        self.author = info['author']
        self.desc = info['desc']

        
    def get_html(self):
        file_path = 'data/episode_list-{webtoon_id}.html'.format(webtoon_id=self.webtoon_id)
        url_episode_list = 'https://comic.naver.com/webtoon/list.nhn'
        params = {
            'titleId': self.webtoon_id,
        }
        if os.path.exists(file_path):
            html = open(file_path, 'rt').read()

        else:
            response = requests.get(url_episode_list, params)
            html = response.text
            open(file_path, 'wt').write(html)
            
        return html
    
    def get_info(self):
        """
        webtoon_id 매개변수를 이용하여
        웹툰 title, author, description을 딕셔너리 형태로 return
        params webtoon_id:
        return: title, author, description 딕셔너리로
        """
        html = self.get_html()
        
        soup = BeautifulSoup(html, 'lxml')

        h2_title = soup.select_one('div.detail > h2')
        title = h2_title.contents[0].strip()
        author = h2_title.contents[1].get_text(strip=True)
        desc = soup.select_one('div.detail > p').get_text()

        webtoon_dict = {
            'title': title,
            'author': author,
            'desc': desc,
        }

        return webtoon_dict

    def episode_crawler(self):
        
        html = self.get_html()
        soup = BeautifulSoup(html, 'lxml')
        
        table = soup.select_one('table.viewList')

        tr_list = table.select('tr')

        for index, tr in enumerate(tr_list[1:]):
            if tr.get('class'):
                continue

            url_thumnail = tr.select_one('td:nth-of-type(1) img').get('src')
            url_detail = tr.select_one('td > a').get('href')

            query_string = parse.urlsplit(url_detail).query
            query_dict = parse.parse_qs(query_string)

            no = query_dict['no'][0]

            title = tr.select_one('td:nth-of-type(2) > a').get_text(strip=True)
            rating = tr.select_one('td:nth-of-type(3) strong').get_text(strip=True)
            created_data = tr.select_one('td:nth-of-type(4)').get_text(strip=True)

        epsisode_dict = {
            'title': title,
            'rating': rating,
            'desc': desc,
        }
        
        return episode_dict

In [20]:
result = Webtoon(718019)

In [21]:
result.title

'옆집친구'

In [11]:
%ls

10minutes_to_pandas.ipynb
20170528204207_6e9df8e618b97520233bbb35e7d4eaaf_IMAG01_1.jpg
20170528204207_6e9df8e618b97520233bbb35e7d4eaaf_IMAG01_2.jpg
20170528204207_6e9df8e618b97520233bbb35e7d4eaaf_IMAG01_3.jpg
Pipfile
Pipfile.lock
Titanic_kaggle.ipynb
bugs.ipynb
bugs_top_100.csv
bugs_top_100.xlsx
crawling_theroy.ipynb
data/
kaggle_titanic_train.csv
lecture.ipynb
weekday.html


In [12]:
query = {
    'titleId': [234234], 'no': [5655653]
}

print(query['no'])

[5655653]


### select_one 과 select 의 차이

- select_one: 요소 하나를 가져옴

- select: 요소의 배열을 모두 가져옴

## 단순하게 생각해서 list 는 get_text( ) 를 사용할 수 없다고 생각하자

### 아래에 members 라는 변수에 list 를 만들었다

#### list 안에 값들을 꺼내오는 가장 일반적인 방법이 무엇일까?

In [13]:
members = ['민수', '영희', '준희']

## for 문을 돌려서 값들을 하나씩 출력하는 방법이다

In [14]:
for member in members:
    print(member)
    print(type(member))

민수
<class 'str'>
영희
<class 'str'>
준희
<class 'str'>


### 그렇기 때문에 데이터타입 `list` 속성은 get_text()를 사용할 수 없다

In [17]:
soup.select_one('a').get_text() # a 태그 하나에 대해서만 text 값을 추출하기 때문에 가능

In [18]:
a_list = soup.select('a')

for menu in a_list:
    print(menu.get_text())

### 태그 선택자

In [ ]:
soup.select_one('span')

### 아이디 선택자

In [ ]:
soup.select_one('#where')

### 클래스 선택자

In [ ]:
soup.select('.blind')